In [ ]:
from math import degrees, radians, sin, asin, cos, sqrt, pi
from queue import SimpleQueue
from decimal import Decimal
from random import randint

: 

In [30]:
def draw(leaf, q, max_order=4):
    ins   = []
    a,b,c = leaf["triplet"]
    gamma = leaf["gamma"]
    x,y   = leaf["origin"]
    rot   = leaf["rotation"]
    sz    = leaf["side"]
    order = leaf["order"]
    alternate = leaf["alternate"]
    random = leaf["random"]
    if order > max_order: 
        return ""
    ins.append(f"translate([{x}, {y}, 0])")
    ins.append(f"rotate([0, 0, {degrees(rot)}])")
    ins.append(f"cuboid([{sz},{sz},5], center=false, fillet=0.3);")
    # leaf a
    beta = asin(b*sin(gamma)/c)
    la = { "order"   : order+1, 
           "triplet" : triplet, 
           "gamma"   : gamma,
           "origin"  : (x + sz*cos(pi/2+rot), y + sz*sin(pi/2+rot)),
           "rotation": rot + beta,
           "side"    : sz * a / c,
           "alternate": alternate, "random": random}
    # leaf b
    xa, ya = la["origin"]
    sza = la["side"]
    ins.append(f"translate([{xa},{ya},0]) cylinder(h=1, d={sza/100});")
    xc, yc = (xa + sz*cos(rot), ya + sz*sin(rot))
    ins.append(f"translate([{xc},{yc},0]) cylinder(h=1, d={sza/100});")
    alfa = asin(a*sin(gamma)/c)
    lb = { "order"   : order+1, 
           "triplet" : triplet, 
           "gamma"   : gamma,
           "origin"  : (xa + sza*cos(rot+beta), ya + sza*sin(rot+beta)),
           "rotation": rot - alfa,
           "side"    : sz * b / c,
           "alternate": alternate, "random": random}
    xb, yb = lb["origin"]
    szb = lb["side"]
    ins.append(f"translate([{xb},{yb},0]) cylinder(h=1, d={sza/100});")
    if random == True:
        #trp = triplets[randint(0,len(triplets)-1)]
        la["triplet"] = triplets[randint(0,len(triplets)-1)]#trp
        lb["triplet"] = triplets[randint(0,len(triplets)-1)]#trp
    if alternate == True and order%2 == 0:
        la["triplet"] = (b,a,c)
        lb["triplet"] = (b,a,c)
    q.put(la)
    q.put(lb)
    return " ".join(ins)

In [31]:
instructions = []
base         = 20
rot          = 0
triplet      = (3, 4, 5)
gamma        = pi/2
# pi/3 for equal squares (1,1,1)
max_order    = 6
triplets     = [(3,4,5),(5,12,13),(8,15,17),(20,21,29),(28,45,53),(33,56,65),(48,55,73),(36,77,85),(39,80,89),(65,72,97)] # (12,35,37),(7,24,25),(16,63,65),(9,40,41),(11,60,61),(13,84,85)
# [(3,4,5),(5,12,13),(8,15,17),(7,24,25),(20,21,29),(12,35,37),(9,40,41),(28,45,53),(11,60,61),(16,63,65),(33,56,65),(48,55,73),(13,84,85),(36,77,85),(39,80,89),(65,72,97)]

leafs = SimpleQueue()
leafs.put({"order":     0, 
           "triplet":   triplet, 
           "gamma":     pi/2, 
           "origin":    (0.0, 0.0), 
           "rotation":  radians(rot), 
           "side":      base, 
           "alternate": True, 
           "random":    False})

# leafs.put({"order":0, "triplet":triplet, "gamma":pi/2, "origin":(0.0, 0.0), "rotation":radians(rot), "side":base, "alternate":True, "random":False})


while True:
    ins = draw(leafs.get(), leafs, max_order=max_order)
    if ins == "":
        break
        
    instructions.append(ins)
                        
#print("\n".join(instructions))

In [32]:
with open("/home/cesar/code/openscad/coasters/hexagonal/pyt_tree_instructions2.scad", "w") as f:
    f.write("\n".join(instructions))